In [1]:
import pandas as pd
import numpy as np

raw_path = "../../Formato_dataset_productos.csv"
df = pd.read_csv(raw_path, low_memory=False)

# Ver primeras filas y columnas
df.head()
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124514 entries, 0 to 124513
Data columns (total 15 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   FECHA_CORTE    124514 non-null  int64  
 1   FECHA_MUESTRA  124514 non-null  int64  
 2   DEPARTAMENTO   124514 non-null  object 
 3   PROVINCIA      124514 non-null  object 
 4   DISTRITO       124514 non-null  object 
 5   UBIGEO         124514 non-null  int64  
 6   ANO            124514 non-null  int64  
 7   MES            124514 non-null  int64  
 8   COD_CULTIVO    124514 non-null  int64  
 9   CULTIVO        124514 non-null  object 
 10  SIEMBRA        124514 non-null  float64
 11  COSECHA        116335 non-null  float64
 12  PRODUCCION     124514 non-null  float64
 13  VERDE_ACTUAL   124512 non-null  float64
 14  PRECIO_CHACRA  124514 non-null  float64
dtypes: float64(5), int64(6), object(4)
memory usage: 14.2+ MB


,FECHA_CORTE,FECHA_MUESTRA,UBIGEO,ANO,MES,COD_CULTIVO,SIEMBRA,COSECHA,PRODUCCION,VERDE_ACTUAL,PRECIO_CHACRA
count,124514.0,1.245140e+05,124514.000000,124514.000000,124514.000000,1.245140e+05,124514.000000,116335.000000,124514.000000,124512.000000,124514.000000
mean,20250527.0,2.019984e+07,200338.110662,2019.861004,201992.693239,1.366054e+10,9.582755,17.217866,151.504984,152.651492,0.473059
std,0.0,2.715068e+04,190.429778,2.715068,271.255753,8.366462e+08,97.202457,205.738878,1716.729435,762.823342,1.255347
min,20250527.0,2.015123e+07,200101.000000,2015.000000,201508.000000,1.201001e+10,0.000000,0.000000,0.000000,0.000000,0.000000
25%,20250527.0,2.018123e+07,200203.000000,2018.000000,201801.000000,1.301023e+10,0.000000,0.000000,0.000000,4.000000,0.000000
50%,20250527.0,2.020123e+07,200305.000000,2020.000000,202006.000000,1.401007e+10,0.000000,0.000000,0.000000,14.000000,0.000000
75%,20250527.0,2.022123e+07,200409.000000,2022.000000,202209.000000,1.404014e+10,0.000000,0.000000,4.000000,60.000000,0.460000
max,20250527.0,2.024123e+07,200806.000000,2024.000000,202412.000000,1.502002e+10,5500.000000,20512.000000,147000.000000,21444.750000,27.000000


In [2]:
# --- Fechas ---
df["FECHA_MUESTRA"] = pd.to_datetime(df["FECHA_MUESTRA"].astype(str), format="%Y%m%d", errors="coerce")

# La columna MES trae yyyymm; la separamos en año y mes con tolerancia a nulos
df["MES_STR"] = df["MES"].astype(str)
df["ANO"] = pd.to_numeric(df["MES_STR"].str[:4], errors="coerce").astype("Int64")
df["MES_NUM"] = pd.to_numeric(df["MES_STR"].str[4:6], errors="coerce").astype("Int64")

# Fecha yyyy-mm (primer día del mes)
df["FECHA_YYYYMM"] = pd.to_datetime(
    df["ANO"].astype(str) + df["MES_NUM"].astype(str).str.zfill(2),
    format="%Y%m", errors="coerce"
)

# --- UBIGEO a 6 dígitos ---
df["UBIGEO"] = df["UBIGEO"].astype(str).str.zfill(6)

# --- Numéricos clave ---
for col in ["SIEMBRA","COSECHA","PRODUCCION","VERDE_ACTUAL","PRECIO_CHACRA"]:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# Producción en kg (PRECIO_CHACRA está en S/ por kg)
df["PRODUCCION_KG"] = df["PRODUCCION"] * 1000

# Rendimiento mensual (t/ha), evitando división por 0
df["REND_MES_THA"] = np.where(df["COSECHA"] > 0,
                                df["PRODUCCION"] / df["COSECHA"],
                                np.nan)

In [3]:
# Control de calidad
# 1) % de fechas mensuales válidas
pct_fecha = df["FECHA_YYYYMM"].notna().mean()
print(f"% filas con FECHA_YYYYMM válida: {pct_fecha:.1%}")

# 2) UBIGEO bien formateado
assert df["UBIGEO"].str.len().eq(6).all(), "UBIGEO no tiene 6 dígitos"

# 3) ¿Cuántos casos 'producción>0 & cosecha=0'? (desfase esperado)
casos_desfase = ((df["PRODUCCION"]>0) & (df["COSECHA"]==0)).sum()
print("Filas con PRODUCCION>0 y COSECHA=0:", casos_desfase)


% filas con FECHA_YYYYMM válida: 100.0%
Filas con PRODUCCION>0 y COSECHA=0: 20900


In [4]:
# Derivar año y trimestre
df["ANIO"] = df["FECHA_YYYYMM"].dt.year
df["TRIM"] = df["FECHA_YYYYMM"].dt.quarter

tri_dist = (df.groupby(["UBIGEO","ANIO","TRIM"], as_index=False)
            .agg(PROD_SUM=("PRODUCCION","sum"),
                COSE_SUM=("COSECHA","sum"),
                PRECIO_MEAN=("PRECIO_CHACRA","mean"),
                PRECIO_COUNT=("PRECIO_CHACRA", lambda s: s.notna().sum()),
                PROD_KG_SUM=("PRODUCCION_KG","sum")))

tri_dist["REND_THA_TRIM"] = np.where(tri_dist["COSE_SUM"]>0,
                                        tri_dist["PROD_SUM"]/tri_dist["COSE_SUM"],
                                        np.nan)
tri_dist.head()

,UBIGEO,ANIO,TRIM,PROD_SUM,COSE_SUM,PRECIO_MEAN,PRECIO_COUNT,PROD_KG_SUM,REND_THA_TRIM
0,200101,2015,3,1563.000,146.0,0.405397,63,1563000.0,10.705479
1,200101,2015,4,23208.322,1776.0,0.581875,96,23208322.0,13.067749
2,200101,2016,1,6595.795,286.0,0.529326,89,6595795.0,23.062220
3,200101,2016,2,9117.430,521.0,0.766702,94,9117430.0,17.499866
4,200101,2016,3,3854.770,131.0,0.464950,101,3854770.0,29.425725


In [5]:
# Anual por distrito
anual_dist = (df.groupby(["UBIGEO","ANIO"], as_index=False)
                .agg(PROD_SUM=("PRODUCCION","sum"),
                      COSE_SUM=("COSECHA","sum"),
                      PRECIO_MEAN=("PRECIO_CHACRA","mean"),
                      PRECIO_COUNT=("PRECIO_CHACRA", lambda s: s.notna().sum()),
                      PROD_KG_SUM=("PRODUCCION_KG","sum")))

anual_dist["REND_THA_ANUAL"] = np.where(anual_dist["COSE_SUM"]>0,
                                        anual_dist["PROD_SUM"]/anual_dist["COSE_SUM"],
                                        np.nan)
anual_dist.head()

,UBIGEO,ANIO,PROD_SUM,COSE_SUM,PRECIO_MEAN,PRECIO_COUNT,PROD_KG_SUM,REND_THA_ANUAL
0,200101,2015,24771.322,1922.0,0.511950,159,24771322.0,12.888305
1,200101,2016,38524.195,3015.0,0.638462,377,38524195.0,12.777511
2,200101,2017,54249.755,2875.0,0.574192,365,54249755.0,18.869480
3,200101,2018,51626.700,3209.0,0.462166,374,51626700.0,16.088096
4,200101,2019,37765.580,2550.0,0.464652,374,37765580.0,14.810031


In [14]:
# Bandera mensual
df["_has_prod"]  = df["PRODUCCION"].fillna(0) > 0
df["_price_ok"]  = df["PRECIO_CHACRA"].notna() & (df["PRECIO_CHACRA"] > 0)

# Cobertura por distrito-año
cov_precio = (df.groupby(["UBIGEO","ANIO"], as_index=False)
                .agg(
                    MESES_CON_PROD=("_has_prod", "sum"),
                    MESES_CON_PRECIO_OK=("_price_ok", "sum")
                ))

cov_precio["PRICE_COVERAGE"] = np.where(
    cov_precio["MESES_CON_PROD"] > 0,
    cov_precio["MESES_CON_PRECIO_OK"]/cov_precio["MESES_CON_PROD"],
    1.0
)
cov_precio["ALL_PRICE_OK"] = cov_precio["PRICE_COVERAGE"].eq(1.0)

num_filas_false = (cov_precio["ALL_PRICE_OK"] == False).sum()
print("Filas con ALL_PRICE_OK = False:", num_filas_false)


cov_precio[cov_precio["ALL_PRICE_OK"] == False]

Filas con ALL_PRICE_OK = False: 0


,UBIGEO,ANIO,MESES_CON_PROD,MESES_CON_PRECIO_OK,PRICE_COVERAGE,ALL_PRICE_OK
